In [3]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error

In [3]:
pd.__version__


'1.4.2'

In [12]:
!pip install pyarrow

In [4]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
#Duration
df['duration']= df.tpep_dropoff_datetime - df.tpep_pickup_datetime #create a column

df.duration=df.duration.apply(lambda td: td.total_seconds() / 60)
# Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive)
df[((df.duration >= 1) & (df.duration <= 60))]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [5]:
#df.head()
#td = df.duration.iloc[0]
#td.total_seconds()/60 #in minutes
#sns.distplot(df.duration)
#df.duration.describe(percentiles=[0.95, 0.98, 0.99])
#((df.duration >= 1) & (df.duration <= 60)).mean()

In [6]:
sns.distplot(y_pred, label= 'prediction')
sns.distplot(y_train, label= 'actual')

plt.legend()
plt.xlim(-30,100)


NameError: name 'y_pred' is not defined

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    #Duration
    df['duration']= df.tpep_dropoff_datetime - df.tpep_pickup_datetime #create a column
    df.duration=df.duration.apply(lambda td: td.total_seconds() / 60)
    
    # Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive)
    df[((df.duration >= 1) & (df.duration <= 60))]

    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [12]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [14]:
target = 'duration'
y_train = df_train[target].values
y_val= df_val[target].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_train, y_pred, squared = False)

ValueError: Found input variables with inconsistent numbers of samples: [3066766, 2913955]